In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [30]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai "langchain-google-community[featurestore]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.147.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [41]:
from google.cloud import aiplatform
import google.cloud.bigquery as bq
import langchain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.schema import format_document
from langchain_community.document_loaders import DirectoryLoader, TextLoader,BigQueryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore
from datetime import datetime
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore



PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [42]:
# Define our query
query = """
SELECT id,media_type,content,test_metadata 
FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
"""

# Load the data
loader = BigQueryLoader(
    query, metadata_columns=["id"], page_content_columns=["content","media_type","test_metadata"]
)


/var/tmp/ipykernel_710571/1683864506.py:8: LangChainDeprecationWarning: The class `BigQueryLoader` was deprecated in LangChain 0.0.32 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import BigQueryLoader``.
  loader = BigQueryLoader(


In [52]:
DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}


embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)


store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


### source

https://cloud.google.com/blog/products/ai-machine-learning/rag-with-bigquery-and-langchain-in-cloud

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []
documents.extend(loader.load())
 
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5#,
   # separators=["\n\n"],
)
doc_splits = text_splitter.split_documents(documents)


# get current processing time to add it to metadata, datetime object containing current date and time
now = datetime.now()

# Add chunk number to metadata
chunk_idx=0
prev=doc_splits[0].metadata["id"]
for idx, split in enumerate(doc_splits):
    split.metadata["process_time"]=now
    if prev==split.metadata["id"]:
       split.metadata["chunk"] = chunk_idx      
    else:
        chunk_idx=0
        split.metadata["chunk"] = chunk_idx
        prev=split.metadata["id"]
    chunk_idx +=1
        

In [55]:

DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}



embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
) 

_=bq_store.add_documents(doc_splits)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


In [57]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore,BigQueryLoader
from datetime import datetime
import os
import logging
import json
    
def chunk_and_embedding(project_id: str= None, dataset: str= None, table: str= None, region: str =None,\
                        metadata_columns: list[str]=None, page_content_columns: list[str]= None, \
                        source_query_str: str= None, separators:  list[str]=None, chunk_size: int=None, \
                       chunk_overlap: int=0):
    """
        Chunks the combination of page_content_columns from a given bigquery source string and generates
        text embeddings for them.

        Args:
            

        Returns:
             
        """
    

     # Load the data
    loader = BigQueryLoader(
        source_query_str, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )

    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)

    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")


    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()

    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    for idx, split in enumerate(doc_splits):
        split.metadata["process_time"]=now
        if prev==split.metadata["id"]:
           split.metadata["chunk"] = chunk_idx      
        else:
            chunk_idx=0
            split.metadata["chunk"] = chunk_idx
            prev=split.metadata["id"]
        chunk_idx +=1

        rows_to_insert.append(
                           {"id": split.metadata["id"], 
                               "process_time":split.metadata["process_time"].isoformat(),
                               "content":  split.page_content,
                               "chunk": split.metadata["chunk"]
                              }
                                     )
    logging.info (f"Metadata added to chunks")

    from google.cloud import bigquery

    client = bigquery.Client()
    table_id = f"{project_id}.{dataset}.{table}"

 
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print("New rows have been added.")
    else:
        print(f"Encountered errors: {errors}")
    
    
    
#     logging.info (f"Bigquery store info is set -  ProjectID {project_id}, Region {region}, Dataset {dataset}, Table {table}")


#     _=bq_store.add_documents(doc_splits)
    
#     logging.info (f"Chunks and embeddings added to the store")

    return 'done'

if __name__ == "__main__":
    project_id= os.environ.get("PROJECT_ID") 
    dataset= os.environ.get("DATASET")  
    table= os.environ.get("TABLE") 
    region= os.environ.get("REGION") 
    metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
    page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
    source_query_str= os.environ.get("SOURCE_QUERY_STR") 
    separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
    chunk_size= os.environ.get("CHUNK_SIZE")
    chunk_overlap= 0 if os.environ.get("CHUNK_OVERLAP")=="" else os.environ.get("CHUNK_OVERLAP")
 

    project_id= 'nine-quality-test' 
    dataset= 'my_langchain_dataset'
    table= 'doc_and_vectors'
    region= 'us-central1'
    metadata_columns= "id".split(",")
    page_content_columns= "content,media_type,test_metadata".split(',') 
    source_query_str= """
    SELECT id,media_type,content,test_metadata 
    FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
    """
    separators= "\n\n"
    chunk_size= 25
    chunk_overlap= 5
    
    
    message=chunk_and_embedding(project_id=project_id, dataset=dataset, table=table, region=region,\
                        metadata_columns= metadata_columns, page_content_columns=page_content_columns, \
                        source_query_str=source_query_str, separators=separators, chunk_size=chunk_size, \
                       chunk_overlap=chunk_overlap)
    print(message)
     

New rows have been added.
done


In [12]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.4 MB/s eta 0:00:0000:01


In [17]:
import googleapiclient.discovery


def list_sub_directories(bucket_name, prefix):
    """Returns a list of sub-directories within the given bucket."""
    service = googleapiclient.discovery.build('storage', 'v1')

    req = service.objects().list(bucket=bucket_name, prefix=prefix, delimiter='/')
    
    res = req.execute()
    
    return res['prefixes']

# For the example (gs://abc/xyz), bucket_name is 'abc' and the prefix would be 'xyz/'
#print(list_sub_directories(bucket_name='raw_nine_files/sub_dir', prefix=''))

In [20]:
list(list_sub_directories(bucket_name=bucket, prefix=prefix))

['2023/', '2024/']

In [24]:
bucket='raw_nine_files'
prefix=''
l=[]
for main in list(list_sub_directories(bucket_name=bucket, prefix=prefix)):
    for dircty in list(list_sub_directories(bucket_name=bucket, prefix=main )):
       for sub_dircty in list(list_sub_directories(bucket_name=bucket, prefix=dircty )):
           print(sub_dircty)
        

2023/1/a/
2023/1/b/
2023/2/c/
2024/3/d/
